In [61]:

import pandas as pd
from yahoo_fin import stock_info as si
from bqplot import DateScale, LinearScale, Axis, Lines, Figure
from bqplot.interacts import (FastIntervalSelector, IndexSelector,
                             BrushIntervalSelector, BrushSelector, MultiSelector)
from ipywidgets import ToggleButtons, VBox, HTML, SelectMultiple, Dropdown, HBox, Output, Checkbox
import numpy as np
import yfinance as yf

# Robin's Simple Stock Viewer

Use the dropdown box on the bottom left to select a ticker symbol.  This should contain all ticker symbols from NASDAQ that don't have rights issues, warrants, bankruptcy or first preferred status. 
(Aside:  The financial package I am using should allow me to get symbols from the S&P 500 and the Dow, as well as others, but it isn't working at the moment -- I had to stick with NASDAQ).  The middle dropdown allows selection of the type of data to display for the selected company.  The checkbox toggles the display (below the widgets) of the income statement from Yahoo financial data for the company.  (There are many, *many* possible financial items that could be displayed instead). Allow 5-15 seconds for the data to load, as it is actively scraped from websites as selected.

The plot defaults to Amazon (AMZN) data, with 'adjusted close' value as the plotted data.  

Click in the plot area to activate a data selector: moving the mouse up or down in the plot area with the selector 'active' will adjust the width of the selector.  Clicking again will lock the width, but still allow horizontal positioning of the selector.

At the top of the plot, the selector reports how many days of data have been selected, the change in value from start to finish, and the % change for the selected interval.  (For demonstration purposes only; any statistic of the data could be reported here).





In [62]:
ticker_list = si.tickers_nasdaq()
# exclude 5-char symbols for 'Rights issues', 'Warrants', 'Q bankruptcy', and 'Preferred first'
# see https://levelup.gitconnected.com/how-to-get-all-stock-symbols-a73925c16a1b
exclude = ['W', 'R', 'P', 'Q']
removed = []
saved = []
for symbol in ticker_list:
    if len(symbol) > 4 and symbol[-1] in exclude:
        removed.append(symbol)
    else:
        saved.append(symbol)

In [63]:
symbol = 'amzn'
data = si.get_data(symbol)

In [64]:
data_types = data.columns.to_list()[:-1]
dates_actual = data.index.values
dt_x_fast = DateScale()
lin_y = LinearScale()
prices = data['adjclose'].values
x_ax = Axis(label='Date', scale=dt_x_fast)
x_ay = Axis(label=(symbol + ' Price'), scale=lin_y, orientation='vertical')
lc = Lines(x=dates_actual, y = prices, scales = {'x': dt_x_fast, 'y': lin_y}, colors = ['blue'])


In [65]:
intsel_fast = FastIntervalSelector(scale=dt_x_fast, marks=[lc])

In [66]:

def date_interval_callback(change):
    try:
        start = change.new[0]
        end = min(change.new[-1], len(lc.x))

        days = (lc.x[end] - lc.x[start]).astype('timedelta64[D]') / np.timedelta64(1, 'D')

        inc = lc.y[end] - lc.y[start]
        perc_inc = inc * 100 / lc.y[start]

        db_date.value = 'Over %d days increase of %.2f or %.2f%%' % (int(days), inc, perc_inc)
    except IndexError:
        db_date.value = "no data yet"
    
   
def ticker_callback(change):
    #global data
    outbox.clear_output()
    ticker = selTicker.value
    data = si.get_data(ticker)
    data_type = selType.value
    lc.y = data[data_type].values
    x_ay.label = "# shares" if data_type == "volume" else "%s Price ($)" % ticker.upper()
    info = yf.Ticker(ticker).info
    fig_fast_intsel.title = "%s in %s sector" % (info['shortName'],
                                                info['sector'])
    if check_info.value:
        with outbox:
            display(si.get_income_statement(ticker))
    


In [67]:
# widgets
db_date = HTML()
db_date.value = "Starting value"

fig_fast_intsel = Figure(marks=[lc], axes=[x_ax, x_ay], title='Amazon Data',
                         interaction=intsel_fast) #This is where we assign the interaction to this particular Figure

selTicker = Dropdown(options = saved, value = 'AMZN', description = 'ticker')
selType = Dropdown(options = data_types, value = 'adjclose', description = 'type')
check_info = Checkbox(value=False, description = "Show Income Statement")


# connections
lc.observe(date_interval_callback, names = ['selected'])
selTicker.observe(ticker_callback, names = 'value')
selType.observe(ticker_callback, names = 'value')
check_info.observe(ticker_callback, names = 'value')

# arrange output
lastline = HBox([selTicker, selType, check_info])
firstline = HBox([db_date])
outbox = Output(layout = {'border': '1px solid black'})
VBox([firstline, fig_fast_intsel, lastline, outbox])

In [ ]:
%%html
<style>
div.input {
    display:none;
}
</style>